<a href="https://colab.research.google.com/github/melekbairamova20/melekbairamova20/blob/main/Crack_Segmentatiton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and preprocess dataset

In [ ]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt

In [ ]:
!wget https://raw.githubusercontent.com/databyhuseyn/DeepLearning/refs/heads/main/helper_functions.py

--2024-11-02 13:49:49--  https://raw.githubusercontent.com/databyhuseyn/DeepLearning/refs/heads/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-11-02 13:49:50 (73.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
from helper_functions import plot_loss_curves, unzip_data

In [ ]:
unzip_data('/content/Conglomerate Concrete Crack Detection.zip')

FileNotFoundError: [Errno 2] No such file or directory: '/content/Conglomerate Concrete Crack Detection.zip'

In [ ]:
IMG_SIZE = 128, 128
BATCH_SIZE = 8

train_images_dir = '/content/Conglomerate Concrete Crack Detection/Train/images'
train_masks_dir = '/content/Conglomerate Concrete Crack Detection/Train/masks'
test_images_dir  = '/content/Conglomerate Concrete Crack Detection/Test/images'
test_masks_dir = '/content/Conglomerate Concrete Crack Detection/Test/masks'

def load_data(image_dir, mask_dir, img_size):
  images, masks = [], []

  for filename in os.listdir(image_dir):
    img = load_img(os.path.join(image_dir, filename), target_size = img_size)
    img = img_to_array(img) / 255.0

    mask = load_img(os.path.join(mask_dir, filename), target_size = img_size, color_mode = "grayscale")
    mask = img_to_array(mask) / 255.0
    masks.append(mask)

  return np.array(images), np.array(masks)

X_train, y_train = load_data(
    train_images_dir,
    train_masks_dir,
    IMG_SIZE
)

X_test, y_test = load_data(
    test_images_dir,
    test_masks_dir,
    IMG_SIZE
)

print(f'Train images shape: {X_train.shape}, Train masks shape: {y_train.shape}')
print(f'Test images shape: {X_test.shape}, Test masks shape: {y_test.shape}')


NameError: name 'os' is not defined

# Data Visualization

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

train_images_dir = '/content/Conglomerate Concrete Crack Detection/Train/images'
train_masks_dir = '/content/Conglomerate Concrete Crack Detection/Train/masks'

image_files = os.listdir(train_images_dir)

fig, axes = plt.subplot(nrows = 3, ncols = 3, figsize = (12, 12))

for i in range(9):
  n = np.random.randint(len(image_files))
  image_path = os.path.join(train_images_dir, image_files[n])
  mask_path = os.path.join(train_masks_dir, image_files[n])

  img = plt.imread(image_path)
  mask = plt.imread(mask_path)

  ax = axes[i // 3, i % 3]
  ax.imshow(img, cmpay = 'gray')
  ax.imshow(mask, cmap = 'jet', alpha = 0.35)

  ax.axis('off')

plt.tight_layout()
plt.show()


# Create Model

In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()

def create_unet_model(input_shape=(128, 128, 3)):

  inputs = tf.keras.layers.Input(shape=input_shape)
  conv2d_0 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
  conv2d_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2d_0)            # birleshdir
  max_pool_0 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(conv2d_1)
  conv2d_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(max_pool_0)
  conv2d_3 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2d_2)            # birleshdir
  max_pool_1 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(conv2d_3)
  conv2d_4 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(max_pool_1)
  conv2d_5 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2d_4)            # birleshdir
  max_pool_2 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(conv2d_5)
  conv2d_6 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(max_pool_2)
  conv2d_7 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2d_6)            # birleshdir
  max_pool_3 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(conv2d_7)
  conv2d_8 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(max_pool_3)
  conv2d_9 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2d_8)
  up_conv_0 = tf.keras.layers.Conv2DTranspose(filters=512, kernel_size=(2, 2), strides=(2, 2), padding='same')(conv2d_9)
  up_conv_0_resized = tf.keras.layers.Lambda(lambda x: tf.image.resize(x, size=(up_conv_0.shape[1], up_conv_0.shape[2])))(conv2d_7)

  concat_0 = tf.keras.layers.concatenate([up_conv_0_resized, up_conv_0], axis=3)
  conv2d_10 = tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(concat_0)
  conv2d_11 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2d_10)
  up_conv_1 = tf.keras.layers.Conv2DTranspose(filters = 256, kernel_size=(2, 2), strides=(2, 2), padding='same')(conv2d_11)
  up_conv_1_resized = tf.keras.layers.Lambda(lambda x: tf.image.resize(x, size=(up_conv_1.shape[1], up_conv_1.shape[2])))(conv2d_5)

  concat_1 = tf.keras.layers.concatenate([up_conv_1_resized, up_conv_1], axis=3)
  conv2d_12 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(concat_1)
  conv2d_13 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2d_12)
  up_conv_2 = tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=(2, 2), strides=(2, 2), padding='same')(conv2d_13)
  up_conv_2_resized = tf.keras.layers.Lambda(lambda x: tf.image.resize(x, size=(up_conv_2.shape[1], up_conv_2.shape[2])))(conv2d_3)

  concat_2 = tf.keras.layers.concatenate([up_conv_2_resized, up_conv_2], axis=3)
  conv2d_14 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(concat_2)
  conv2d_15 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2d_14)
  up_conv_3 = tf.keras.layers.Conv2DTranspose(filters = 64, kernel_size=(2, 2), strides=(2, 2), padding='same')(conv2d_15)
  up_conv_3_resized = tf.keras.layers.Lambda(lambda x: tf.image.resize(x, size=(up_conv_3.shape[1], up_conv_3.shape[2])))(conv2d_1)

  concat_3 = tf.keras.layers.concatenate([up_conv_3_resized, up_conv_3], axis=3)
  conv2d_16 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(concat_3)
  conv2d_17 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2d_16)
  conv2d_18 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2d_17)
  outputs = tf.keras.layers.Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid')(conv2d_18)

  unet_model = tf.keras.Model(inputs, outputs, name='Unet-remake')
  return unet_model

In [ ]:
input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3)
input_shape

(128, 128, 3)

In [ ]:
unet_model = create_unet_model(input_shape = input_shape)

In [ ]:
unet_model.summary()

In [ ]:
tf.keras.utils.plot_model(unet_model, show_shapes = True)

# Let's fit the model

In [ ]:
model_ckp = tf.keras.callbacks.ModelCheckpoint('/content/best_model.weights.h5',
                                               save_best_only = True,
                                               save_weights_only = True)
early_stopping = tf.keras.callbacks.EarlyStopping(restore_best_weights = True,
                                                  patience = 3)

In [ ]:
unet_model.compile(loss = 'binary_crossentropy',
                   optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3),
                   metrics = ["accuracy"])


In [ ]:
history = unet_model.fit(X_train, y_train
                         epochs = 8,
                         validation_data = (X_test, y_test),
                         callbacks = [model_ckp, early_stoppint])

In [ ]:
img_path = '/content/cracked_concrete.jpg'
new_img = load_img(img_path, target_size = IMG_SIZE)
new_img = img_to_array(new_img) / 255.0

In [ ]:
pred = unet_model(tf.expand_dims(new_img, axis = 0))

In [ ]:
original_img = '/content/cracked_concrete.jpg'
or_img = plt.imread(original_img)
or_img = load_img(original_img, target_size = IMG_SIZE)
or_img = img_to_array(or_img) / 255.0

plt.subplot(121)
plt.imshow(or_img)
plt.title('Original Image')
plt.subplot(122)
plt.imshow(tf.squeeze(pred), cmap = 'binary')
plt.title('Predicted Segmentation')
plt.show()